In [1]:
import os
os.environ['AIOD_RAIL_API_KEY'] = '9b9c7f0cd6af7656b2b480d842d79863'

In [2]:
import json
import aiod_rail_sdk
import aiod_rail_sdk.configuration
from client.client import RailClient


In [3]:
script_path = '/home/eddie/Projects/script.py'
requirements_path = '/home/eddie/Projects/requirements.txt'
image_path = '/home/eddie/Projects/Dockerfile'


In [4]:
with open('/home/eddie/Projects/template.json') as f:
  template = json.load(f)

Create the client

In [5]:
config = aiod_rail_sdk.configuration.Configuration(host='http://localhost/api')
my_client = RailClient(config, api_key=os.environ.get('AIOD_RAIL_API_KEY'))

Experiment Templates endpoints

Create experiment template from json file

In [6]:
resp = my_client.experiments_templates.create_experiment_template(file=template)
resp

ExperimentTemplateResponse(name='ExpTempl', description='Template description', task=<TaskType.TEXT_CLASSIFICATION: 'TEXT_CLASSIFICATION'>, datasets_schema=AssetSchema(cardinality=<AssetCardinality.ENUM_1_MINUS_1: '1-1'>), models_schema=AssetSchema(cardinality=<AssetCardinality.ENUM_1_MINUS_1: '1-1'>), envs_required=[EnvironmentVarDef(name='SPLIT_NAME', description='name of a subset')], envs_optional=[], script='import os\n\nos.environ["HF_HOME"] = "."\n\nimport json\nimport logging\n\nimport numpy as np\nimport sklearn.metrics as m\nimport torch\nimport wandb\nfrom datasets import load_dataset\nfrom tqdm import tqdm\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\n\n\ndef get_device():\n    return "cuda" if torch.cuda.is_available() else "cpu"\n\n\ndef wandb_check():\n    necessary_envs = [\n        "WANDB_API_KEY",\n        "WANDB_ENTITY",\n        "WANDB_PROJECT",\n        "WANDB_NAME",\n    ]\n    return set(necessary_envs).issubset(os.environ)\n\n\ndef 

Update the experiment

In [8]:
template_changed = template.copy()
template_changed['name'] = 'CreatedExperiment'
template_changed['description'] = 'DescriptionChangedToSomethingElse'
template_changed

{'name': 'CreatedExperiment',
 'description': 'DescriptionChangedToSomethingElse',
 'task': 'TEXT_CLASSIFICATION',
 'datasets_schema': {'cardinality': '1-1'},
 'models_schema': {'cardinality': '1-1'},
 'envs_required': [{'name': 'SPLIT_NAME', 'description': 'name of a subset'}],
 'envs_optional': [],
 'available_metrics': ['accuracy'],
 'public': True,
 'base_image': 'python:3.9',
 'script': 'import os\r\n\r\nos.environ["HF_HOME"] = "."\r\n\r\nimport json\r\nimport logging\r\n\r\nimport numpy as np\r\nimport sklearn.metrics as m\r\nimport torch\r\nimport wandb\r\nfrom datasets import load_dataset\r\nfrom tqdm import tqdm\r\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\r\n\r\n\r\ndef get_device():\r\n    return "cuda" if torch.cuda.is_available() else "cpu"\r\n\r\n\r\ndef wandb_check():\r\n    necessary_envs = [\r\n        "WANDB_API_KEY",\r\n        "WANDB_ENTITY",\r\n        "WANDB_PROJECT",\r\n        "WANDB_NAME",\r\n    ]\r\n    return set(necessary_en

Let's check the experiment templates, which are not approved and not built and take the first one from them to select it for update

In [17]:
resp = my_client.experiments_templates.get(finalized=False, approved=False)
templates = [i.model_dump() for i in resp]
experiment_template_id = templates[0]['id']
print(f'id: {experiment_template_id},\ntemplate: {templates[0]}')

id: 6655b12972f875012fe205f5,
template: {'name': 'ExpTempl', 'description': 'Template description', 'task': <TaskType.TEXT_CLASSIFICATION: 'TEXT_CLASSIFICATION'>, 'datasets_schema': {'cardinality': <AssetCardinality.ENUM_1_MINUS_1: '1-1'>}, 'models_schema': {'cardinality': <AssetCardinality.ENUM_1_MINUS_1: '1-1'>}, 'envs_required': [{'name': 'SPLIT_NAME', 'description': 'name of a subset'}], 'envs_optional': [], 'script': 'import os\n\nos.environ["HF_HOME"] = "."\n\nimport json\nimport logging\n\nimport numpy as np\nimport sklearn.metrics as m\nimport torch\nimport wandb\nfrom datasets import load_dataset\nfrom tqdm import tqdm\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\n\n\ndef get_device():\n    return "cuda" if torch.cuda.is_available() else "cpu"\n\n\ndef wandb_check():\n    necessary_envs = [\n        "WANDB_API_KEY",\n        "WANDB_ENTITY",\n        "WANDB_PROJECT",\n        "WANDB_NAME",\n    ]\n    return set(necessary_envs).issubset(os.environ

Let's update the selected experiment template, we will check the experiment id to verify that it's the same one we updated

In [35]:
updated_experiment_template = my_client.experiments_templates.update(experiment_template_id, file=template_changed)
id = dict(updated_experiment_template)['id']
print(f'id: {id},\ntemplate: {updated_experiment_template}')


id: 6655b12972f875012fe205f5,
template: name='CreatedExperiment' description='DescriptionChangedToSomethingElse' task=<TaskType.TEXT_CLASSIFICATION: 'TEXT_CLASSIFICATION'> datasets_schema=AssetSchema(cardinality=<AssetCardinality.ENUM_1_MINUS_1: '1-1'>) models_schema=AssetSchema(cardinality=<AssetCardinality.ENUM_1_MINUS_1: '1-1'>) envs_required=[EnvironmentVarDef(name='SPLIT_NAME', description='name of a subset')] envs_optional=[] script='import os\n\nos.environ["HF_HOME"] = "."\n\nimport json\nimport logging\n\nimport numpy as np\nimport sklearn.metrics as m\nimport torch\nimport wandb\nfrom datasets import load_dataset\nfrom tqdm import tqdm\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\n\n\ndef get_device():\n    return "cuda" if torch.cuda.is_available() else "cpu"\n\n\ndef wandb_check():\n    necessary_envs = [\n        "WANDB_API_KEY",\n        "WANDB_ENTITY",\n        "WANDB_PROJECT",\n        "WANDB_NAME",\n    ]\n    return set(necessary_envs).is

Let's visualize experiment which are not approved and not built again

In [41]:
resp = my_client.experiments_templates.get(finalized=False, approved=False)
templates = [i.model_dump() for i in resp]
experiments = [(i['id'], i['name']) for i in templates]
experiments

[('6655b12972f875012fe205f5', 'CreatedExperiment'),
 ('6655b24a72f875012fe205f6', 'ExperimentChanged'),
 ('66570433b986f37c731ead6c', 'ExpTempl')]

Select the last from these experiments and approve it

In [60]:
my_client.experiments_templates.approve_experiment_template(experiments[2][0], is_approved=True)

We can see that the updated experiment temlpate has dissappeared

In [61]:
response = my_client.experiments_templates.get(finalized=False, approved=False)
rest_of_e_t = [i.model_dump() for i in response]
for t in rest_of_e_t:
    if t['approved'] == False:
        print(f'{t}')

{'name': 'CreatedExperiment', 'description': 'DescriptionChangedToSomethingElse', 'task': <TaskType.TEXT_CLASSIFICATION: 'TEXT_CLASSIFICATION'>, 'datasets_schema': {'cardinality': <AssetCardinality.ENUM_1_MINUS_1: '1-1'>}, 'models_schema': {'cardinality': <AssetCardinality.ENUM_1_MINUS_1: '1-1'>}, 'envs_required': [{'name': 'SPLIT_NAME', 'description': 'name of a subset'}], 'envs_optional': [], 'script': 'import os\n\nos.environ["HF_HOME"] = "."\n\nimport json\nimport logging\n\nimport numpy as np\nimport sklearn.metrics as m\nimport torch\nimport wandb\nfrom datasets import load_dataset\nfrom tqdm import tqdm\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\n\n\ndef get_device():\n    return "cuda" if torch.cuda.is_available() else "cpu"\n\n\ndef wandb_check():\n    necessary_envs = [\n        "WANDB_API_KEY",\n        "WANDB_ENTITY",\n        "WANDB_PROJECT",\n        "WANDB_NAME",\n    ]\n    return set(necessary_envs).issubset(os.environ)\n\n\ndef process

Let's get the count of experiment templates which are not approved and not built

In [64]:
my_client.experiments_templates.count(finalized=False, approved=False)

2

Let's get the first experiment by it's id

In [65]:
e_t = rest_of_e_t[0]
my_client.experiments_templates.get_by_id(e_t['id'])

ExperimentTemplateResponse(name='CreatedExperiment', description='DescriptionChangedToSomethingElse', task=<TaskType.TEXT_CLASSIFICATION: 'TEXT_CLASSIFICATION'>, datasets_schema=AssetSchema(cardinality=<AssetCardinality.ENUM_1_MINUS_1: '1-1'>), models_schema=AssetSchema(cardinality=<AssetCardinality.ENUM_1_MINUS_1: '1-1'>), envs_required=[EnvironmentVarDef(name='SPLIT_NAME', description='name of a subset')], envs_optional=[], script='import os\n\nos.environ["HF_HOME"] = "."\n\nimport json\nimport logging\n\nimport numpy as np\nimport sklearn.metrics as m\nimport torch\nimport wandb\nfrom datasets import load_dataset\nfrom tqdm import tqdm\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\n\n\ndef get_device():\n    return "cuda" if torch.cuda.is_available() else "cpu"\n\n\ndef wandb_check():\n    necessary_envs = [\n        "WANDB_API_KEY",\n        "WANDB_ENTITY",\n        "WANDB_PROJECT",\n        "WANDB_NAME",\n    ]\n    return set(necessary_envs).issubset

We will archive this experiment template

In [70]:
my_client.experiments_templates.archive(e_t['id'], archived=True)

Let's look again on count and list of experiments which are not approved and not built

In [72]:
my_client.experiments_templates.count(finalized=False, approved=False)

2

In [77]:
resp = my_client.experiments_templates.get(finalized=False, approved=False)
templates = [i.model_dump() for i in resp]
for t in templates:
    print(f"Name: {t['name']} ID: {t['id']}\nArchived: {t['archived']}" )


Name: CreatedExperiment ID: 6655b12972f875012fe205f5
Archived: True
Name: ExperimentChanged ID: 6655b24a72f875012fe205f6
Archived: False


We can also view number of datasets we can browse

In [54]:
my_client.datasets.count()

411932

Let's get two of them

In [78]:
my_client.datasets.get(offset=0, limit=2)

[Dataset(platform='huggingface', platform_resource_identifier='acronym_identification', name='acronym_identification', date_published=datetime.datetime(2022, 3, 2, 23, 29, 22), same_as='https://huggingface.co/datasets/acronym_identification', is_accessible_for_free=True, version=None, issn=None, measurement_technique=None, temporal_coverage=None, ai_asset_identifier=2, ai_resource_identifier=2, aiod_entry=AIoDEntryRead(editor=[], status='published', date_modified=datetime.datetime(2023, 12, 19, 10, 25, 29), date_created=datetime.datetime(2023, 12, 19, 10, 25, 29)), alternate_name=[], application_area=[], citation=[], contact=[], creator=[], description=Text(plain='Acronym identification training and development sets for the acronym identification task at SDU@AAAI-21.', html=None), distribution=[Distribution(platform=None, platform_resource_identifier=None, checksum=None, checksum_algorithm=None, copyright=None, content_url='https://huggingface.co/datasets/acronym_identification/resolve

Load json for experiment to create in this json it is important to change the id of template to experiment template id from which we want to create the experiment,
We will select the one which we approved => ('66570433b986f37c731ead6c', 'ExpTempl'), we can change it right here in code as needed, let's also change it's name

In [82]:
with open('/home/eddie/Projects/exp_TODO_ADD_TEMPLATE_ID.json') as f:
    experiment = json.load(f)

print(experiment)
experiment['experiment_template_id'] = '66570433b986f37c731ead6c'
experiment['name'] = 'TodayExperiment'
print(experiment)


{'name': 'Exp01', 'description': 'Exp description', 'publication_ids': [], 'experiment_template_id': '6655ae4172f875012fe205f4', 'dataset_ids': ['1'], 'model_ids': ['2'], 'metrics': ['accuracy'], 'env_vars': [{'key': 'SPLIT_NAME', 'value': 'train'}], 'public': True}
{'name': 'TodayExperiment', 'description': 'Exp description', 'publication_ids': [], 'experiment_template_id': '66570433b986f37c731ead6c', 'dataset_ids': ['1'], 'model_ids': ['2'], 'metrics': ['accuracy'], 'env_vars': [{'key': 'SPLIT_NAME', 'value': 'train'}], 'public': True}


Create experiment from provided json file

In [83]:
resp = my_client.experiments.create_experiment(file=experiment)
resp

ExperimentResponse(name='TodayExperiment', description='Exp description', experiment_template_id='66570433b986f37c731ead6c', publication_ids=[], dataset_ids=['1'], model_ids=['2'], env_vars=[EnvironmentVar(key='SPLIT_NAME', value='train')], public=True, id='6657103fb986f37c731ead6d', created_at=datetime.datetime(2024, 5, 29, 11, 23, 42, 208857, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2024, 5, 29, 11, 23, 42, 208854, tzinfo=TzInfo(UTC)), archived=False, mine=True)

Let's display all experiments, we can see our experiment TodayExperiment as the last listed

In [85]:
resp = my_client.experiments.get(archived=False)
resp

[ExperimentResponse(name='Exp01', description='Exp description', experiment_template_id='6655ae4172f875012fe205f4', publication_ids=[], dataset_ids=['1'], model_ids=['2'], env_vars=[EnvironmentVar(key='SPLIT_NAME', value='train')], public=True, id='6655d24f72f875012fe205f9', created_at=datetime.datetime(2024, 5, 28, 12, 47, 10, 518000), updated_at=datetime.datetime(2024, 5, 28, 12, 47, 10, 518000), archived=False, mine=True),
 ExperimentResponse(name='Exp01', description='Exp description', experiment_template_id='6655ae4172f875012fe205f4', publication_ids=[], dataset_ids=['1'], model_ids=['2'], env_vars=[EnvironmentVar(key='SPLIT_NAME', value='train')], public=True, id='6655d51372f875012fe205fa', created_at=datetime.datetime(2024, 5, 28, 12, 58, 58, 771000), updated_at=datetime.datetime(2024, 5, 28, 13, 4, 36, 529000), archived=True, mine=True),
 ExperimentResponse(name='TodayExperiment', description='Exp description', experiment_template_id='66570433b986f37c731ead6c', publication_ids=

We can also list only the number and list only those which are archived

In [88]:
resp = my_client.experiments.count(archived=True)
print(f'Count: {resp}\nExperiment{my_client.experiments.get(archived=True)}')

Count: 1
Experiment[ExperimentResponse(name='Exp01', description='Exp description', experiment_template_id='6655ae4172f875012fe205f4', publication_ids=[], dataset_ids=['1'], model_ids=['2'], env_vars=[EnvironmentVar(key='SPLIT_NAME', value='train')], public=True, id='6655d51372f875012fe205fa', created_at=datetime.datetime(2024, 5, 28, 12, 58, 58, 771000), updated_at=datetime.datetime(2024, 5, 28, 13, 4, 36, 529000), archived=True, mine=True)]


We can now run the experiment which we will select by providing it's id, lets select the one which we created before 

In [103]:
resp = my_client.experiments.get(archived=False)
experiments = [i.model_dump() for i in resp]
id_of_experiment = [i['id'] for i in experiments if i['name'] == 'TodayExperiment']
id_of_experiment

['6657103fb986f37c731ead6d']

In [105]:
resp = my_client.experiments.run_experiment(id=id_of_experiment[0])
resp

ExperimentRunResponse(id='6657130bb986f37c731ead6e', created_at=datetime.datetime(2024, 5, 29, 11, 35, 39, 575983, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2024, 5, 29, 11, 35, 39, 575995, tzinfo=TzInfo(UTC)), retry_count=0, state=<RunState.CREATED: 'CREATED'>, metrics={}, archived=False, public=True, mine=True, experiment_id='6657103fb986f37c731ead6d')

We can display number of experiment runs and it's count

In [107]:
resp = my_client.experiments.get_experiments_run_count(id=id_of_experiment[0])
resp

1

In [114]:
resp = my_client.experiments.get_experiments_run(id=id_of_experiment[0])
id_of_run = dict(resp[0])['id']
id_of_run

'6657130bb986f37c731ead6e'

Let's display logs from the run which we executed on the experiment

In [123]:
resp = my_client.experiments.logs_experiment_run(id=id_of_run)

from pprint import pprint

pprint(resp)

('{"workflow_logs": "2024-05-29 11:36:00,160 | root | MainThread | INFO | '
 'Publishing step:0, cmd: set -a && source .env && set +a && python script.py, '
 'total steps 1 to MQ\\n2024-05-29 11:40:12,901 | root | MainThread | INFO | '
 'Workflow 682f339f-6679-4c0f-a6e5-536786724684 finished. Files available at '
 '/var/reana/users/00000000-0000-0000-0000-000000000000/workflows/682f339f-6679-4c0f-a6e5-536786724684.\\n\\n", '
 '"job_logs": {"5782b3a9-a2d7-4782-a1e9-1cc77a39af60": {"workflow_uuid": '
 '"682f339f-6679-4c0f-a6e5-536786724684", "job_name": "Execute Python script", '
 '"compute_backend": "Kubernetes", "backend_job_id": '
 '"reana-run-job-332e98a7-8e4d-4474-96c5-78c9510294e7", "docker_img": '
 '"docker.io/marveso/rail-exp-templates:template-66570433b986f37c731ead6c", '
 '"cmd": "set -a && source .env && set +a && python script.py", "status": '
 '"finished", "logs": "job: :\\n '
 '/usr/local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: '
 'FutureWarning: 

We can also download files from experiment, this call will download script.py into the folder where this code is located

In [128]:
my_client.experiments.download_experiment_run(id=id_of_run, filepath='script.py', to_dir='')

Let's archive our experiment

In [129]:
resp = my_client.experiments.archive(id=id_of_experiment[0], archived=True)

In [131]:
resp = my_client.experiments.get(archived=True)
resp

[ExperimentResponse(name='Exp01', description='Exp description', experiment_template_id='6655ae4172f875012fe205f4', publication_ids=[], dataset_ids=['1'], model_ids=['2'], env_vars=[EnvironmentVar(key='SPLIT_NAME', value='train')], public=True, id='6655d51372f875012fe205fa', created_at=datetime.datetime(2024, 5, 28, 12, 58, 58, 771000), updated_at=datetime.datetime(2024, 5, 28, 13, 4, 36, 529000), archived=True, mine=True),
 ExperimentResponse(name='TodayExperiment', description='Exp description', experiment_template_id='66570433b986f37c731ead6c', publication_ids=[], dataset_ids=['1'], model_ids=['2'], env_vars=[EnvironmentVar(key='SPLIT_NAME', value='train')], public=True, id='6657103fb986f37c731ead6d', created_at=datetime.datetime(2024, 5, 29, 11, 23, 42, 208000), updated_at=datetime.datetime(2024, 5, 29, 11, 58, 37, 332000), archived=True, mine=True)]

We can also delete the run which we executed earlier but before that, we have to un-archive the experiment to be able to delete it

In [133]:
resp = my_client.experiments.archive(id=id_of_experiment[0], archived=False)

In [134]:
my_client.experiments.delete_experiment_run(id=id_of_run)

As we can see the run is deleted

In [136]:
resp = my_client.experiments.get_experiments_run_count(id=id_of_experiment[0])
resp

0